# Quick False Positive Identification (LC-only)

This is a fast, deterministic first demo: we generate a synthetic eclipsing-binary-like signal and show how LC-only checks (odd/even depth, secondary eclipse, shape) quickly flag it.

**No network, no TPFs, no catalogs.**

In [1]:
import numpy as np

from tess_vetter.api import Candidate, Ephemeris, LightCurve, vet_candidate

rng = np.random.default_rng(42)

## Build a synthetic EB-like light curve

We simulate:
- a **primary eclipse** with alternating depth (odd/even mismatch), and
- a **secondary eclipse** at phase 0.5.

This is intentionally "obvious" so you can see the pipeline working in one run.

In [2]:
# Time grid (days)
t_start = 0.0
t_stop = 30.0
cadence_min = 2.0
time = np.arange(t_start, t_stop, cadence_min / (60.0 * 24.0), dtype=np.float64)

# EB-like ephemeris
period_days = 2.0
t0_btjd = 0.3
duration_hours = 3.0
duration_days = duration_hours / 24.0

# Primary eclipse: alternate depth by transit index (odd/even)
depth_odd = 0.05  # 5%
depth_even = 0.02  # 2%

# Secondary eclipse depth (at phase 0.5)
secondary_depth = 0.015  # 1.5%

phase = ((time - t0_btjd) / period_days) % 1.0
dist_primary = np.minimum(phase, 1.0 - phase) * period_days
dist_secondary = np.minimum(np.abs(phase - 0.5), 1.0 - np.abs(phase - 0.5)) * period_days

in_primary = dist_primary <= (duration_days / 2.0)
in_secondary = dist_secondary <= (duration_days / 2.0)

# Determine primary event index (0-based), then odd/even classification
event_index = np.floor((time - t0_btjd) / period_days + 0.5).astype(int)
is_odd = (event_index % 2) == 0  # label: first event is "odd"

flux = np.ones_like(time)
flux[in_primary & is_odd] -= depth_odd
flux[in_primary & (~is_odd)] -= depth_even
flux[in_secondary] -= secondary_depth

# Add small Gaussian noise
flux_err = np.full_like(time, 5e-4)
flux = flux + rng.normal(0.0, flux_err, size=flux.shape)

lc = LightCurve(time=time, flux=flux, flux_err=flux_err)
candidate = Candidate(
    ephemeris=Ephemeris(period_days=period_days, t0_btjd=t0_btjd, duration_hours=duration_hours),
    depth_ppm=50_000.0,
)

bundle = vet_candidate(lc, candidate, network=False, checks=["V01", "V02", "V05"])

## Inspect the results

For this synthetic EB-like signal you should see:
- a clear odd/even mismatch (V01)
- a detectable secondary (V02)
- shape hints consistent with an eclipse-like event (V05)

Exact numbers will vary with noise, but the overall pattern should be stable.

In [3]:
print("Vetting Results (LC-only quick FP demo)")
print("=" * 45)
for r in bundle.results:
    conf = r.confidence
    conf_str = f"{conf:.3f}" if conf is not None else "None"
    print(f"{r.id} {r.name}: status={r.status} confidence={conf_str}")
    if r.flags:
        print(f"  flags: {r.flags}")
    if r.metrics:
        # Print a few metrics if present (keys differ by check)
        keys = sorted(r.metrics.keys())[:6]
        for k in keys:
            print(f"  {k}: {r.metrics[k]}")

Vetting Results (LC-only quick FP demo)
V01 Odd-Even Depth: status=ok confidence=0.700
  delta_ppm: -30059.5
  delta_sigma: 431.19
  depth_diff_sigma: 431.19
  depth_err_even_ppm: 62.2
  depth_err_odd_ppm: 31.6
  depth_even_ppm: 50028.8
V02 Secondary Eclipse: status=ok confidence=0.900
  baseline_flux: 0.999991
  n_baseline_points: 6485
  n_secondary_events_effective: 15
  n_secondary_points: 6471
  red_noise_inflation: 1.41
  secondary_depth: 0.00014
V05 V-Shape: status=ok confidence=0.935
  depth_bottom: 0.049172
  depth_edge: 0.049129
  depth_ppm: 35995.3
  method: trapezoid_grid_search
  n_baseline: 8760
  n_bottom_points: 345


## Takeaway

This is the simplest and most decisive style of vetting: LC-only checks can quickly eliminate many eclipsing-binary false positives without pixel data or catalog queries.

Next: see the periodogram tutorial (finding candidates) and the blend/localization tutorial (host ambiguity).